In [5]:
import os
from dotenv import load_dotenv

from couchbase.exceptions import CouchbaseException
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions

import json
from typing import Optional, List, Any, Dict
from datetime import timedelta
import traceback

load_dotenv()

True

In [6]:
endpoint = "couchbases://cb.82kuz4rgjdzyhlh.cloud.couchbase.com"
cb_username = os.getenv("CB_USER")
cb_password = os.getenv("CB_PASSWORD")
cb_bucketname = os.getenv("BUCKET_NAME")
cb_scope = os.getenv("SCOPE_NAME")
cb_collection = "Test_DB"

In [8]:
from pymongo import MongoClient

# Connect to local MongoDB
client = MongoClient("mongodb://localhost:27017/")

# Choose DB and collection
db = client["testDB"]
collection = db["test_coll"]

# Insert a document
collection.insert_one({"hello": "world", "count": 1})

InsertOneResult(ObjectId('685ebdca37b8cbff637eb0b3'), acknowledged=True)

In [ ]:
def write_db(document: Dict[str, Any]) -> None:

    key = document["key"]
    auth = PasswordAuthenticator(cb_username, cb_password) # type: ignore
    options = ClusterOptions(auth) 
    options.apply_profile("wan_development")

    try:
        
        cluster = Cluster(endpoint, options) # type: ignore

    except:

        print('WRITING INTO LOCAL DATABASE')

        document["_id"] = document.pop("key")
        client = MongoClient("mongodb://localhost:27017/")
        db = client["testDB"]
        collection = db["test_coll"]

        collection.insert_one(document)

        return None

    try:

        print("SENDING")

        # Wait until the cluster is ready for use.
        cluster.wait_until_ready(timedelta(seconds=5))

        # Get a reference to our bucket
        cb = cluster.bucket(cb_bucketname)

        # Get a reference to our collection
        cb_coll = cb.scope(cb_scope).collection(cb_collection) # type: ignore

        # Simple K-V operation - to create a document with specific ID
        try:
            result = cb_coll.insert(key, document)
            print("\nCreate document success. CAS: ", result.cas)
        except CouchbaseException as e:
            print(e)

    except Exception as e:

        print(e)

In [ ]:
# define document
test_doc = {"key": "123", "message": "Jordan is better than LeBron"}

write_db(test_doc)

WRITING INTO LOCAL DATABASE


DuplicateKeyError: E11000 duplicate key error collection: testDB.test_coll index: _id_ dup key: { _id: "787fbdca37b8cbff637ebsc3" }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: testDB.test_coll index: _id_ dup key: { _id: "787fbdca37b8cbff637ebsc3" }', 'keyPattern': {'_id': 1}, 'keyValue': {'_id': '787fbdca37b8cbff637ebsc3'}}